In [64]:
import numpy as np
from datetime import datetime
from datetime import timedelta
import sys
import re
import os
import itertools
import glob
import pandas as pd

In [127]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!ls /content/drive/My\ Drive/FYP/Synchronized/bitf21m541

clean_the_table				fall_left		  slow_walk
downstair				fall_right		  standing
fall_backward				fast_walk		  standing_up_from_laying
fall_backward_while_trying_to_sit_down	jogging			  talk_using_phone
fall_backward_while_trying_to_stand_up	laying			  typing
fall_forward				laying_down_from_sitting  upstair
fall_forward_when_trying_to_sit_down	reading			  walking
fall_forward_while_trying_to_stand_up	sitting


In [66]:
def prep_data(feat_filename, lab_cls, target_length):
    # Multiply target_length by 4 as per your original logic.
    target_length = target_length * 4
    wholeList = []

    try:
        X = pd.read_csv(feat_filename, delimiter=",", decimal='.')
    except Exception as e:
        print(f"Error reading {feat_filename}: {e}")
        return [], []

    # If the file has 4 or more columns, use only the first 4 columns.
    if len(X.columns) >= 4:
        X = X.iloc[:, :4]  # Take only the first 4 columns
        X.columns = ['Timestamp', 'X', 'Y', 'Z']
    elif len(X.columns) == 3:
        X.columns = ['X', 'Y', 'Z']
    else:
        print(f"Unexpected number of columns in {feat_filename}.")
        return [], []


    # Check if there are enough samples in the X channel.
    if len(X['X']) >= target_length:
        # Extract the first target_length samples from each channel.
        new_list1 = X['X'][0:target_length].tolist()
        new_list2 = X['Y'][0:target_length].tolist()
        new_list3 = X['Z'][0:target_length].tolist()
        # Concatenate the channels into one feature vector.
        wholeList.append(new_list1 + new_list2 + new_list3)
        Y = [lab_cls]
        # print(f"Loaded {feat_filename}: extracted {target_length} samples per channel.")
        return wholeList, Y
    else:
        # print(f"File {feat_filename} does not have enough samples (needed: {target_length}).")
        return [], []

In [70]:
def load_data(filename, target_length):
    fname_lower = filename.lower()
    
    # Determine class based on activity keyword in the filename.
    if 'bending' in fname_lower:
        cls = 0
    elif 'clean_the_table' in fname_lower:
        cls = 1
    elif 'close_door' in fname_lower:
        cls = 2
    elif 'close_lid_by_rotation' in fname_lower:
        cls = 3
    elif 'downstairs' in fname_lower:
        cls = 4
    elif 'drink_water' in fname_lower:
        cls = 5
    elif 'eat_small_thing' in fname_lower:
        cls = 6
    elif 'standing_up_from_laying' in fname_lower:
        cls = 7
    elif 'jogging' in fname_lower:
        cls = 8
    elif 'laying_down_from_sitting' in fname_lower:
        cls = 9
    elif 'standing_up_from_sitting' in fname_lower:
        cls = 10
    elif 'open_bag' in fname_lower:
        cls = 11
    elif 'open_big_box' in fname_lower:
        cls = 12
    elif 'open_door' in fname_lower:
        cls = 13
    elif 'pick_from_floor' in fname_lower:
        cls = 14
    elif 'plugin' in fname_lower:
        cls = 15
    elif 'put_on_floor' in fname_lower:
        cls = 16
    elif 'reading' in fname_lower:
        cls = 17
    elif 'sitting_down_from_standing' in fname_lower:
        cls = 18
    elif 'sitting' in fname_lower:
        cls = 19
    elif 'slow_walk' in fname_lower:
        cls = 20
    elif 'squatting' in fname_lower:
        cls = 21
    elif 'quick_walk' in fname_lower: 
        cls = 22
    elif 'laying' in fname_lower:
        cls = 23
    elif 'standing' in fname_lower:
        cls = 24
    elif 'talk_using_phone' in fname_lower:
        cls = 25
    elif 'throw_out' in fname_lower:
        cls = 26
    elif 'typing' in fname_lower:
        cls = 27
    elif 'upstairs' in fname_lower:
        cls = 28
    elif 'walking' in fname_lower:
        cls = 29
    else:
        return [], []
    # print("loading data 🚀🚀🚀🚀🚀🚀")
    # print(filename)
    # print(cls)
    return prep_data(filename, cls, target_length)

In [9]:
def load_data(filename, target_length):
    fname_lower = filename.lower()

    # Fall-related activities only
    if 'fall_backward' in fname_lower and 'trying_to_sit_down' in fname_lower:
        cls = 0
    elif 'fall_backward' in fname_lower and 'trying_to_stand_up' in fname_lower:
        cls = 1
    elif 'fall_backward' in fname_lower:
        cls = 2
    elif 'fall_forward' in fname_lower and 'trying_to_sit_down' in fname_lower:
        cls = 3
    elif 'fall_forward' in fname_lower and 'trying_to_stand_up' in fname_lower:
        cls = 4
    elif 'fall_forward' in fname_lower:
        cls = 5
    elif 'fall_left' in fname_lower:
        cls = 6
    elif 'fall_right' in fname_lower:
        cls = 7
    else:
        return [], []

    return prep_data(filename, cls, target_length)


# Check DS Stats

In [94]:
import os
import glob
from collections import defaultdict

# Define sensor list
sensor_order = [
    'glass_accelerometer',
    'glass_gyroscope',
    'glass_magnetometer',
    'phone_accelerometer',
    'phone_gyroscope',
    'phone_magnetometer',
    'watch_accelerometer',
    'watch_gyroscope',
    'watch_magnetometer'
]

# Initialize counters
sensor_file_counts = defaultdict(int)
total_event_instances = 0

# Path to dataset
base_path = r"C:\Users\Malik Haider\Documents\HUMCARE\DS_ADL"

# Traverse all subject folders
subject_folders = glob.glob(os.path.join(base_path, "*"))

for subject in subject_folders:
    if not os.path.isdir(subject):
        continue
    activity_folders = glob.glob(os.path.join(subject, "*"))
    
    for activity_folder in activity_folders:
        if not os.path.isdir(activity_folder):
            continue

        event_files = glob.glob(os.path.join(activity_folder, "*.csv"))
        event_dict = defaultdict(dict)

        for f in event_files:
            filename = os.path.basename(f).lower()
            if "_e" in filename:
                try:
                    event_num = int(filename.split("_e")[1].split(".")[0])
                except Exception:
                    continue

                for sensor in sensor_order:
                    if sensor in filename:
                        sensor_file_counts[sensor] += 1
                        event_dict[event_num][sensor] = f
                        break
        if(sensor_file_counts["watch_accelerometer"] != sensor_file_counts["phone_accelerometer"] or sensor_file_counts["watch_gyroscope"] != sensor_file_counts["watch_accelerometer"]):
            
            print(sensor_file_counts["watch_accelerometer"])
            print(sensor_file_counts["phone_accelerometer"])
            print(activity_folder)
        # Count events where at least one sensor file exists
        total_event_instances += len(event_dict)
   
    
# Display Results
print("📊 Sensor File Counts:")
for sensor in sensor_order:
    print(f"  {sensor:25s}: {sensor_file_counts[sensor]} files")

print(f"\n📦 Total Distinct Event Instances: {total_event_instances}")


📊 Sensor File Counts:
  glass_accelerometer      : 17536 files
  glass_gyroscope          : 17536 files
  glass_magnetometer       : 17536 files
  phone_accelerometer      : 17536 files
  phone_gyroscope          : 17536 files
  phone_magnetometer       : 17536 files
  watch_accelerometer      : 17536 files
  watch_gyroscope          : 17536 files
  watch_magnetometer       : 17536 files

📦 Total Distinct Event Instances: 17531


# Load Data

In [90]:
import os
import glob
import pandas as pd

# ==============================
# Define training arrays for 9 sensors and labels.
# ==============================
GlassAccelerometer_train = []
GlassGyroscope_train = []
GlassMagnetometer_train = []
PhoneAccelerometer_train = []
PhoneGyroscope_train = []
PhoneMagnetometer_train = []
WatchAccelerometer_train = []
WatchGyroscope_train = []
WatchMagnetometer_train = []
Y_train = []

# ==============================
# Define target lengths for each sensor type.
# These represent the number of samples (per channel) to extract.
# Adjust these values based on your data.
# ==============================
target_lengths = {
    'glass_accelerometer': 4,
    'glass_gyroscope': 4,
    'glass_magnetometer': 4,
    'phone_accelerometer': 490, 
    'phone_gyroscope': 490,
    'phone_magnetometer': 100,
    'watch_accelerometer': 98,
    'watch_gyroscope': 98,
    'watch_magnetometer': 98
}

# ==============================
# Define the fixed sensor order.
# This order is used to align events across sensors.
# ==============================
sensor_order = [
    'glass_accelerometer',
    'glass_gyroscope',
    'glass_magnetometer',
    'phone_accelerometer',
    'phone_gyroscope',
    'phone_magnetometer',
    'watch_accelerometer',
    'watch_gyroscope',
    'watch_magnetometer'
]

base_path = r"C:\Users\Malik Haider\Documents\HUMCARE\DS+AF\DS_AF"  # Adjust this path as needed to synchronized and structured base folder

# Get all subject folders under base_path.
subject_folders = glob.glob(os.path.join(base_path, "*"))

# Process each subject folder.
for subject in subject_folders:
    if not os.path.isdir(subject):
        continue
    print(f"🤵🏻🤵🏻🤵🏻 Processing subject: {subject}")

    # Get all activity folders in this subject folder.
    activity_folders = glob.glob(os.path.join(subject, "*"))
    for activity_folder in activity_folders:
        if not os.path.isdir(activity_folder):
            continue
        print(f"🚀🚀🚀 Processing activity: {activity_folder}")

        # Build a dictionary to group files by event number.
        # The dictionary structure will be:
        #    event_dict[event_number][sensor] = file_path
        event_dict = {}
        # Get all CSV event files in the activity folder.
        event_files = glob.glob(os.path.join(activity_folder, "*.csv"))
        for f in event_files:
            filename = os.path.basename(f).lower()
            if "_e" in filename:
                try:
                    # Extract the event number as an integer.
                    part = filename.split("_e")[1].split(".")[0]
                    event_num = int(part)  # This removes any leading zeros.
                except Exception as e:
                    print(f"Error parsing event number in {filename}: {e}")
                    continue
                # Determine the sensor type for this file using sensor_order.
                sensor_found = None
                for sensor in sensor_order:
                    if sensor in filename:
                        sensor_found = sensor
                        break
                if sensor_found is None:
                    # File does not match any of the 9 sensors; skip it.
                    continue
                # Add file to event_dict.
                if event_num not in event_dict:
                    event_dict[event_num] = {}
                event_dict[event_num][sensor_found] = f
        # Process events in sorted order (ensuring event 0, then event 1, etc.)
        for event_num in sorted(event_dict.keys()):
            event_files_dict = event_dict[event_num]
            # For each sensor in our fixed order, load the event file if it exists.
            event_label = None
            for sensor in sensor_order:
                if sensor in event_files_dict:
                    file_path = event_files_dict[sensor]
                    t_length = target_lengths[sensor]
                    data_list, labels = load_data(file_path, t_length)
                    # Append the data to the appropriate training array.
                    if data_list:
                        if sensor == 'glass_accelerometer':
                            GlassAccelerometer_train.extend(data_list)
                        elif sensor == 'glass_gyroscope':
                            GlassGyroscope_train.extend(data_list)
                        elif sensor == 'glass_magnetometer':
                            GlassMagnetometer_train.extend(data_list)
                        elif sensor == 'phone_accelerometer':
                            PhoneAccelerometer_train.extend(data_list)
                        elif sensor == 'phone_gyroscope':
                            PhoneGyroscope_train.extend(data_list)
                        elif sensor == 'phone_magnetometer':
                            PhoneMagnetometer_train.extend(data_list)
                        elif sensor == 'watch_accelerometer':
                            WatchAccelerometer_train.extend(data_list)
                        elif sensor == 'watch_gyroscope':
                            WatchGyroscope_train.extend(data_list)
                        elif sensor == 'watch_magnetometer':
                            WatchMagnetometer_train.extend(data_list)
                        # Assume all files in the same event share the same label.
                        event_label = labels
            if event_label is not None:
                Y_train.extend(event_label)

            min_length = min(
                len(GlassAccelerometer_train),
                len(GlassGyroscope_train),
                len(GlassMagnetometer_train),
                len(PhoneAccelerometer_train),
                len(PhoneGyroscope_train),
                len(PhoneMagnetometer_train),
                len(WatchAccelerometer_train),
                len(WatchGyroscope_train),
                len(WatchMagnetometer_train),
                len(Y_train)
            )

            GlassAccelerometer_train = GlassAccelerometer_train[:min_length]
            GlassGyroscope_train = GlassGyroscope_train[:min_length]
            GlassMagnetometer_train = GlassMagnetometer_train[:min_length]
            PhoneAccelerometer_train = PhoneAccelerometer_train[:min_length]
            PhoneGyroscope_train = PhoneGyroscope_train[:min_length]
            PhoneMagnetometer_train = PhoneMagnetometer_train[:min_length]
            WatchAccelerometer_train = WatchAccelerometer_train[:min_length]
            WatchGyroscope_train = WatchGyroscope_train[:min_length]
            WatchMagnetometer_train = WatchMagnetometer_train[:min_length]
            Y_train = Y_train[:min_length]
print("Total sample for accelerometer:", len(WatchAccelerometer_train))
print("Total label samples after truncation:", len(Y_train))


Total sample for accelerometer: 0
Total label samples after truncation: 0


In [84]:
print('GlassAccelerometer_train:', len(GlassAccelerometer_train))
print('GlassGyroscope_train:', len(GlassGyroscope_train))
print('GlassMagnetometer_train:', len(GlassMagnetometer_train))
print('PhoneAccelerometer_train:', len(PhoneAccelerometer_train))
print('PhoneGyroscope_train:', len(PhoneGyroscope_train))
print('PhoneMagnetometer_train:', len(PhoneMagnetometer_train))
print('WatchAccelerometer_train:', len(WatchAccelerometer_train))
print('WatchGyroscope_train:', len(WatchGyroscope_train))
print('WatchMagnetometer_train:', len(WatchMagnetometer_train))
print('Y_train:', len(Y_train))

GlassAccelerometer_train: 15965
GlassGyroscope_train: 15965
GlassMagnetometer_train: 15965
PhoneAccelerometer_train: 15965
PhoneGyroscope_train: 15965
PhoneMagnetometer_train: 15965
WatchAccelerometer_train: 15965
WatchGyroscope_train: 15965
WatchMagnetometer_train: 15965
Y_train: 15965


In [86]:
unique_labels, counts = np.unique(Y_train, return_counts=True)

In [88]:
print("Unique labels and counts:", unique_labels, counts)
# print(Y_train)

Unique labels and counts: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29] [ 271  945  232  262  231  255  245  244 1003  240  293  251  263  242
  293  229  280  945  282 1060 1082  288 1101 1043 1081  839  246  970
  232 1017]


In [41]:
#Save data json format
import json
# data_dict = {
#     'PhoneAccelerometer_train': PhoneAccelerometer_train,
#     'PhoneGyroscope_train': PhoneGyroscope_train,
#     'PhoneMagnetometer_train': PhoneMagnetometer_train,
#     'WatchAccelerometer_train': WatchAccelerometer_train,
#     'WatchGyroscope_train': WatchGyroscope_train,
#     'WatchMagnetometer_train': WatchMagnetometer_train,
#     'Y_train': Y_train
# }
data_dict = {
    'GlassAccelerometer_train': GlassAccelerometer_train,
    'GlassGyroscope_train': GlassGyroscope_train,
    'GlassMagnetometer_train': GlassMagnetometer_train,
    'PhoneAccelerometer_train': PhoneAccelerometer_train,
    'PhoneGyroscope_train': PhoneGyroscope_train,
    'PhoneMagnetometer_train': PhoneMagnetometer_train,
    'WatchAccelerometer_train': WatchAccelerometer_train,
    'WatchGyroscope_train': WatchGyroscope_train,
    'WatchMagnetometer_train': WatchMagnetometer_train,
    'Y_train': Y_train
}

# Save the data dictionary to a JSON file.
with open(r'C:\Users\Malik Haider\Documents\DS_AF\FALL_DS.json', 'w') as file:
    json.dump(data_dict, file)

print("Training Data Saved Successfully in json File")

Training Data Saved Successfully in json File


In [43]:
json_file_path = r"C:\Users\Malik Haider\Documents\NewHumCare\NewHumCare\AF_Pre\FALL_DS.json"
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
df = pd.DataFrame(data)

In [44]:
df.head()

,PhoneAccelerometer_train,PhoneGyroscope_train,PhoneMagnetometer_train,WatchAccelerometer_train,WatchGyroscope_train,WatchMagnetometer_train,Y_train
0,"[0.48843205, 0.48843205, 0.48843205, 0.4908263...","[-0.01929479, -0.01929479, -0.02173825, -0.022...","[-30.599998, -30.42, -30.0, -29.76, -29.699999...","[9.92636, 9.868899, 9.742006, 9.6845455, 9.607...","[-0.3494149, -0.33475417, -0.31520647, -0.2932...","[-37.44, -37.92, -37.8, -39.12, -39.12, -38.7,...",2
1,"[0.6751855, 0.67279124, 0.6656084, 0.6560313, ...","[-0.0058557545, -0.005244889, -0.0064666197, -...","[-30.599998, -30.42, -30.599998, -30.48, -30.3...","[9.497798, 9.490616, 9.533711, 9.548077, 9.485...","[0.004886922, 0.0036651916, -0.0012217305, -0....","[-38.46, -38.579998, -38.219997, -38.52, -38.2...",2
2,"[2.8252442, 2.6121538, 2.1021733, 1.6664152, 1...","[-0.9960683, -1.0143943, -1.0260007, -1.065096...","[-28.8, -28.439999, -28.439999, -28.439999, -2...","[6.667852, 7.055712, 6.8522053, 6.43322, 6.366...","[1.690875, 1.9315559, 1.8997909, 1.709201, 0.9...","[-38.7, -38.879997, -39.239998, -39.239998, -3...",2
3,"[3.3088877, 3.4214187, 3.5962007, 3.5746522, 3...","[-1.1078566, -1.0473809, -0.98446184, -0.91909...","[-29.34, -28.68, -29.22, -28.98, -29.039999, -...","[8.50181, 7.4842744, 6.9407907, 6.9479733, 7.1...","[-2.2333233, -1.6224581, -1.1716396, -0.714712...","[-41.64, -41.34, -41.16, -40.5, -40.44, -40.14...",2
4,"[1.1396748, 1.1564348, 1.1612233, 1.1803775, 1...","[0.035683084, 0.033850487, 0.033850487, 0.0350...","[-33.0, -33.12, -33.42, -33.36, -33.48, -33.36...","[10.335769, 10.223241, 10.146626, 10.086772, 9...","[0.67073005, 0.6560693, 0.6377433, 0.6401868, ...","[-38.1, -38.04, -38.1, -37.92, -37.559998, -37...",2


In [45]:
import json

json_path = r'C:\Users\Malik Haider\Documents\NewHumCare\NewHumCare\AF_Pre\FALL_DS.json'
with open(json_path, 'r') as file:
    data_dict = json.load(file)

# Print all top-level keys
print("✅ JSON Keys:", data_dict.keys())

# Check if there are other large arrays
for key in data_dict:
    if isinstance(data_dict[key], list):
        print(f"🔹 {key}: {len(data_dict[key])} elements")

# Check memory size for each dataset
import sys
for key in data_dict:
    print(f"📦 {key}: {sys.getsizeof(data_dict[key]) / (1024*1024):.2f} MB")


✅ JSON Keys: dict_keys(['PhoneAccelerometer_train', 'PhoneGyroscope_train', 'PhoneMagnetometer_train', 'WatchAccelerometer_train', 'WatchGyroscope_train', 'WatchMagnetometer_train', 'Y_train'])
🔹 PhoneAccelerometer_train: 2971 elements
🔹 PhoneGyroscope_train: 2971 elements
🔹 PhoneMagnetometer_train: 2971 elements
🔹 WatchAccelerometer_train: 2971 elements
🔹 WatchGyroscope_train: 2971 elements
🔹 WatchMagnetometer_train: 2971 elements
🔹 Y_train: 2971 elements
📦 PhoneAccelerometer_train: 0.02 MB
📦 PhoneGyroscope_train: 0.02 MB
📦 PhoneMagnetometer_train: 0.02 MB
📦 WatchAccelerometer_train: 0.02 MB
📦 WatchGyroscope_train: 0.02 MB
📦 WatchMagnetometer_train: 0.02 MB
📦 Y_train: 0.02 MB


In [49]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load JSON data
json_path = r'C:\Users\Malik Haider\Documents\NewHumCare\NewHumCare\AF_Pre\FALL_DS.json'
with open(json_path, 'r') as file:
    data_dict = json.load(file)

# Extract labels
Y = np.array(data_dict['Y_train'])

# Split indices (80% train, 20% test)
train_idx, test_idx = train_test_split(
    np.arange(len(Y)), test_size=0.2, random_state=42, stratify=Y
)

# Initialize train and test dictionaries
train_dict, test_dict = {}, {}

# Split **all datasets** (not just Y_train)
for key, values in data_dict.items():
    values_array = np.array(values)  # Convert to NumPy array for efficient slicing
    train_dict[key] = values_array[train_idx].tolist()
    test_dict[key] = values_array[test_idx].tolist()

# Save to JSON files
train_json_path = r'C:\Users\Malik Haider\Documents\NewHumCare\NewHumCare\AF_Pre\train_data_fall.json'
test_json_path = r'C:\Users\Malik Haider\Documents\NewHumCare\NewHumCare\AF_Pre\test_data_fall.json'

with open(train_json_path, 'w') as train_file:
    json.dump(train_dict, train_file, indent=4)  

with open(test_json_path, 'w') as test_file:
    json.dump(test_dict, test_file, indent=4)

print("✅ Train and test JSON files saved successfully!")


✅ Train and test JSON files saved successfully!


In [27]:
# For Opening Training Data

import json

json_file_path = "D:/train_data.json"  # Update path if inside a folder

with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Convert test data to DataFrame
import pandas as pd
df_train = pd.DataFrame(data)

df_train.head()

,GlassAccelerometer_train,GlassGyroscope_train,GlassMagnetometer_train,PhoneAccelerometer_train,PhoneGyroscope_train,PhoneMagnetometer_train,WatchAccelerometer_train,WatchGyroscope_train,WatchMagnetometer_train,Y_train
0,"[2910.0, 3262.0, 3281.0, 3638.0, 2816.0, 3026....","[65041.0, 65258.0, 64666.0, 64337.0, 64763.0, ...","[65497.0, 65502.0, 65487.0, 65498.0, 65491.0, ...","[0.76137936, 0.79250497, 0.87630457, 0.8786988...","[-0.7782423, -0.7629707, -0.7452556, -0.735481...","[118.02, 117.96, 118.14, 118.619995, 118.43999...","[10.139444, 10.563218, 11.018116, 11.475409, 1...","[0.1160644, 0.0842994, 0.0842994, 0.035430185,...","[148.92, 148.68, 147.18, 146.81999, 147.06, 14...",20
1,"[4018.0, 4019.0, 4034.0, 4022.0, 4027.0, 4028....","[65340.0, 65462.0, 65511.0, 65335.0, 65378.0, ...","[65376.0, 65379.0, 65380.0, 65387.0, 65376.0, ...","[8.391933, 8.379962, 8.370385, 8.348836, 8.365...","[-0.002443461, -0.0012217305, -0.0036651914, -...","[53.76, 54.0, 53.399998, 53.399998, 52.98, 52....","[-7.5968018, -7.6231384, -7.61835, -7.6829934,...","[-0.02687807, -0.03176499, -0.03176499, -0.036...","[309.18, 309.6, 309.0, 309.12, 308.63998, 307....",17
2,"[4037.0, 4050.0, 4042.0, 4038.0, 3853.0, 4140....","[92.0, 65517.0, 65443.0, 65329.0, 64845.0, 627...","[171.0, 173.0, 178.0, 180.0, 167.0, 169.0, 162...","[2.6791935, 2.6959534, 2.7055306, 2.6983478, 2...","[0.004886922, 0.0012217305, 0.0018325957, 0.00...","[-7.9199996, -7.8599997, -7.8599997, -7.919999...","[3.074155, 3.0310593, 3.0765493, 3.028665, 3.1...","[-0.061086524, -0.061086524, -0.064751714, -0....","[261.96, 261.66, 261.6, 261.12, 261.36, 261.18...",2
3,"[3867.0, 3748.0, 5957.0, 4171.0, 3696.0, 3939....","[5969.0, 6465.0, 1351.0, 616.0, 1350.0, 450.0,...","[65464.0, 65468.0, 65469.0, 65470.0, 65470.0, ...","[-1.1540405, -1.2474172, -1.3024855, -1.376708...","[-1.1307116, -1.1239921, -1.1282681, -1.128268...","[74.22, 74.46, 73.799995, 73.979996, 74.1, 74....","[9.617508, 9.495404, 9.334992, 9.306262, 9.220...","[0.63285637, 0.4166101, 0.4166101, 0.25045475,...","[111.36, 110.159996, 110.22, 109.68, 110.46, 1...",20
4,"[3867.0, 3878.0, 3877.0, 3880.0, 3970.0, 3889....","[65436.0, 65441.0, 56.0, 65485.0, 65516.0, 276...","[65360.0, 65354.0, 65336.0, 65356.0, 65357.0, ...","[-0.4070267, -0.42139238, -0.43096948, -0.4381...","[-0.0018325957, -0.0012217305, -0.00061086525,...","[82.979996, 83.34, 82.74, 82.32, 82.74, 82.74,...","[-3.8905778, -3.9145198, -3.8594532, -3.770867...","[-0.03176499, -0.02565634, -0.02443461, -0.026...","[202.31999, 203.04, 202.98, 203.09999, 202.92,...",6


In [28]:
# For Opening Testing Data

# Path to your test JSON file
test_json_file_path = "D:/test_data.json"  # Update path accordingly

# Load test data
with open(test_json_file_path, 'r') as json_file:
    test_data = json.load(json_file)

# Convert test data to DataFrame
import pandas as pd
df_test = pd.DataFrame(test_data)

# Extract features
X_test_features = [np.array(df_test[column].to_list()) for column in df_test.columns[:-1]]  # Assuming last column is Y_test

# Extract ground truth labels (if available)
y_test = np.array(df_test['Y_train'])  # Update based on your actual test label column

df_test.head()

,GlassAccelerometer_train,GlassGyroscope_train,GlassMagnetometer_train,PhoneAccelerometer_train,PhoneGyroscope_train,PhoneMagnetometer_train,WatchAccelerometer_train,WatchGyroscope_train,WatchMagnetometer_train,Y_train
0,"[3510.0, 3485.0, 3491.0, 3486.0, 3523.0, 3477....","[64624.0, 65202.0, 0.0, 65044.0, 64757.0, 24.0...","[65478.0, 65483.0, 65474.0, 65475.0, 65475.0, ...","[-0.749408, -0.7446195, -0.74701375, -0.730253...","[0.013439035, 0.015882498, 0.015271631, 0.0134...","[77.159996, 76.92, 76.799995, 76.92, 77.22, 76...","[5.985504, 6.093243, 6.215348, 6.4499793, 6.49...","[0.6169739, 0.6035349, 0.6035349, 0.60231316, ...","[137.22, 136.92, 137.52, 136.8, 136.8, 136.739...",1
1,"[3947.0, 3869.0, 3858.0, 3781.0, 4247.0, 3247....","[830.0, 937.0, 545.0, 3763.0, 3118.0, 4024.0, ...","[65370.0, 65367.0, 65356.0, 65352.0, 65358.0, ...","[5.9378014, 5.9808984, 5.935407, 5.7630196, 5....","[-0.90346974, -0.9083566, -0.9309586, -0.94378...","[85.5, 85.259995, 85.619995, 85.32, 85.56, 85....","[10.618284, 9.742006, 8.937555, 8.432379, 8.21...","[0.44959682, 0.47403142, 0.47403142, 0.4691445...","[279.72, 280.13998, 279.66, 279.96, 280.19998,...",29
2,"[3343.0, 3365.0, 3327.0, 3328.0, 3360.0, 3348....","[65503.0, 65467.0, 65480.0, 153.0, 12.0, 65379...","[27.0, 9.0, 26.0, 22.0, 23.0, 20.0, 23.0, 34.0...","[0.81644773, 0.818842, 0.81644773, 0.84996754,...","[-0.18020524, -0.17898352, -0.17654005, -0.170...","[59.94, 60.18, 59.52, 60.0, 60.48, 60.66, 61.2...","[0.30885202, 0.29688102, 0.2753332, 0.37349546...","[-0.040317107, -0.07452556, -0.07452556, -0.08...","[131.7, 131.81999, 132.42, 131.64, 132.3, 132....",27
3,"[465.0, 5573.0, 54.0, 3413.0, 10651.0, 2623.0,...","[1963.0, 3421.0, 64070.0, 62683.0, 2029.0, 392...","[65455.0, 65463.0, 65465.0, 65469.0, 65471.0, ...","[-1.2091088, -1.5993756, -2.205127, -2.5858169...","[3.1716123, 3.0182853, 2.9364293, 2.7721064, 2...","[64.439995, 64.619995, 64.799995, 64.799995, 6...","[-4.7668557, -4.3598413, -4.015076, -3.5889084...","[-0.42760566, -0.42760566, -0.6768387, -0.6768...","[117.18, 117.899994, 118.74, 118.2, 118.979996...",8
4,"[2651.0, 3878.0, 3755.0, 4042.0, 3765.0, 3805....","[519.0, 1903.0, 1331.0, 954.0, 809.0, 2040.0, ...","[36.0, 46.0, 46.0, 64.0, 52.0, 61.0, 57.0, 59....","[0.18435916, 0.31365, 0.38547826, 0.51716334, ...","[-0.19853121, -0.2144137, -0.21991149, -0.2449...","[123.78, 123.71999, 123.36, 122.7, 122.579994,...","[9.0716305, 9.186552, 9.3421755, 9.416395, 9.5...","[0.86132, 0.74281216, 0.74281216, 0.689056, 0....","[126.18, 126.119995, 127.2, 127.68, 128.04, 12...",20
